In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as path

In [ ]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"

In [ ]:
# csv_name_list = os.listdir(data_path)

In [ ]:
# print(csv_name_list)

In [ ]:
# for csv_name in csv_name_list:
#     print("===================={}===================".format(csv_name))
#     df = pd.read_csv(path.join(data_path, csv_name))
#     print(df.describe())

In [ ]:
def print_isna(df):
    print(pd.isna(df).sum())

# 학습 데이터 셋 만들기

# INPUT

> 소득
    - INC || INCOME, HOUSE_INCOME <========== tn_traveller_master_여행객 Master_C 

    특이사항 - 소득 값이 두가지 존재

    INCOMDE사용으로 일단 대체
    
> 미션
    - MIS || TRAVEL_MISSION(==TRAVEL_PURPOSE) <========== tn_travel_여행_C

    특이사항 - 여행 미션내 값들이 여러개 존재

    3개로 중복 없이

> 여행동반자관계
    - TCR || REL_CD <========== tn_companion_info_동반자정보_C

    기본값 0

> 숙소유형 
    - HTY || LODGING_TYPE_CD <========= tn_visit_area_info_방문지정보_C


> 성별
    - GEN || GENDER  <========== tn_traveller_master_여행객 Master_C

    특이사항 남, 여로 적혀 있음

    해당 갑은 남 : 1, 여 : 0으로 변경

    null 값은 2 : 성별 불명 로 대체
    
    -> 기본값 2

> 여행동기
    - TMT || TRAVEL_MOTiVE_1 ~ 3 <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~3번 존재

    3가지 중복 없이 입력

> 여행스타일
    - STY || TRAVEL_STYL_1 ~ 8  <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~8번이 존재

    => 1개만 받을 예정
    학습 시 max값으로 대체

> 동반자연령대
    - AGE || COMPANION_AGE_GRP <============ tn_companion_info_동반자정보_C

    기본값 0
    

#################################### 여기 까지는 후에 타 서버에서 받아서 예측시 입력해 줄 데이터 #####################################



#################################################################################################################################

# OUTPUT
 

> (DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~

> 여행지 만족도

    train 간에는 모델은 입력된 여행 데이터를 바탕으로 각 여행지의 점수를 맞추는 작업을 수행한다.
        
        출력한 점수 중 상위 k개의 장소를 선정 해당 장소 중에 정답이 존재한다면
    
    predict의 경우 모든 여행지에 대해서 점수를 매기는 작업 수행 -> softmax를 통해서 표준화하고 내림차순 정렬



> 여행 ID  ======>>>>>> 필요 X
    - || TRAVEL_ID <======= tn_traveller_master_여행객 Master_C

> 방문지 주소 # 전처리 후 리스트해서 전달
    - - || LOTNO_ADDR <====== tn_visit_area_info_방문지정보_C

    몇 항목만 사용
        VIS
            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            13: 체험 활동 관광지

            보류
                11:식당/카페
                

-------------------------------------------------------

            제거
                10:상점
                12:기타
                24:숙소
                21:집
                22:친구/친구집
                23:사무실
                

2가지 경우가 존재

1. travel id를 확률을 맞추게하고

    2. 확률 분포를 이용 해당 분포 순으로 해당 id가 방문한 지역 전달

2. 방문지 수에 맞게 데이터를 확장한다.

    2. 이 경우 추가적인 데이터를 심어주는게 좋을 듯하다 예를 들어 순서 방문 순서 정보나 재방문 점수 같은 정보를 추가해서 유저 정보 + 다른 추가 정보 이용 학습


In [ ]:
input_data_csv_name_list = ["tn_traveller_master_여행객 Master_C", "tn_travel_여행_C", "tn_companion_info_동반자정보_C", "tn_visit_area_info_방문지정보_C"]

In [ ]:
output_data_csv_name_list = ["tn_travel_여행_C","tn_visit_area_info_방문지정보_C"]

In [ ]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"
extender = '.csv'

index_dict = {'TRAVEL_ID':['1']}
# input_dict = {'여행객':[],'여행':[],'동반자정보':[],'방문지정보':[]}
# 방문지 정보는 output에서 같이 처리
input_dict = {'여행객':[],'여행':[],'동반자정보':[]}
output_dict = {'TRAVELER_ID':[],'방문지정보':[]}
print(index_dict)
print(input_dict)
print(output_dict)
print(data_path)

In [ ]:
tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))

index_list = tmp_df['TRAVELER_ID'].values
index_dict['TRAVEL_ID'] = index_list
print(index_dict)
# print(tmp_df['TRAVELER_ID'].values)

In [ ]:
get_list = ['TRAVELER_ID','INCOME', 'GENDER','HOUSE_INCOME', 'TRAVEL_MOTIVE_1','TRAVEL_MOTIVE_2','TRAVEL_MOTIVE_3', 
            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))
tmp_value = tmp_df['TRAVELER_ID'].values

tmp_value = [ s[0]+"_"+s for s in tmp_value ]
tmp_df['TRAVEL_ID'] = tmp_value
tmp_df = tmp_df.drop('TRAVELER_ID', axis='columns')

tmp_df = tmp_df.set_index("TRAVEL_ID")

input_dict['여행객'] = tmp_df[get_list[1:]]
print(input_dict['여행객'].columns)
print(input_dict['여행객'].head())
print_isna(input_dict['여행객'])


In [ ]:
get_list = ['TRAVEL_ID', 'TRAVEL_MISSION']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])

input_dict['여행'] = tmp_df[get_list[1:]]
print_isna(input_dict['여행'])

In [ ]:
get_list = ['TRAVEL_ID', 'REL_CD', 'COMPANION_AGE_GRP']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[2]+extender))
tmp_df = tmp_df.set_index(get_list[0])
input_dict['동반자정보'] = tmp_df[get_list[1:]]
print_isna(input_dict['동반자정보'])

In [ ]:
# get_list = ['TRAVEL_ID', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD']

# tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[3]+extender))
# tmp_df = tmp_df.set_index(get_list[0])
# input_dict['방문지정보'] = tmp_df[get_list[1:]]
# # print(tmp_df)

"""output_dict의 방문지정보에서 같이 처리"""

In [ ]:
get_list = ['TRAVEL_ID','TRAVELER_ID']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[0]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['TRAVELER_ID'] = tmp_df[get_list[1:]]
# print(output_dict['TRAVELER_ID'])
print_isna(output_dict['TRAVELER_ID'])

In [ ]:
get_list = ['TRAVEL_ID', 'VISIT_AREA_ID', 'LOTNO_ADDR','LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD','REVISIT_YN','DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'X_COORD','Y_COORD']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['방문지정보'] = tmp_df[get_list[1:]]
print_isna(output_dict['방문지정보'])
# print(tmp_df)

In [ ]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.index)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


In [ ]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
for keys in output_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = output_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


# 방분지 정보 정제

> 아래는 해당 번호들 제외하고 걸러야한다

VIS - VISIT_AREA_TYPE_CD 사용할 항목들

            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            11: 식당 카페
            13: 체험 활동 관광지

            


> 아래는 계산해서 SCORE로 추가

(DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~5


> 최종 11352개의 방문지 존재 해당 정보를 바탕으로 데이터 수립 

    TRAVERL_ID에 해당 하는 정보들을 추가 적으로 붙여넣어 준다.



In [ ]:
tmp_df = output_dict['방문지정보']
print(tmp_df.head())

new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,11,13])]
new_df.loc[:,'SCORE'] = ((new_df['DGSTFN'] + new_df['REVISIT_INTENTION'] + new_df['RCMDTN_INTENTION']) / 3.0).round(4)
# print(new_df)
output_dict['방문지정보'] = new_df
# print(output_dict['방문지정보'])

duplicated_index = new_df.index.duplicated()
print(duplicated_index)

In [ ]:
# tmp_df = input_dict['방문지정보']
# print(tmp_df.head())

# new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,13])]
# # print(new_df)
# new_df = new_df.drop("VISIT_AREA_TYPE_CD", axis='columns')
# input_dict['방문지정보'] = new_df
# # print(output_dict['방문지정보'])

# duplicated_index = new_df.index.duplicated()
# print(duplicated_index)

# 데이터 이어 붙이기

나머지 정보들을 output_dict['방문자정보']에 적절한 client_code에 이어 붙여 준다.


여행객
----------------------------------------
Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['TRAVEL_MISSION'], dtype='object')
--------------------------------------------------------------------------------
동반자정보
----------------------------------------
Index(['REL_CD', 'COMPANION_AGE_GRP'], dtype='object')
--------------------------------------------------------------------------------
방문지정보
----------------------------------------
Index(['LODGING_TYPE_CD'], dtype='object')
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
TRAVELER_ID
----------------------------------------
Index(['TRAVELER_ID'], dtype='object')
--------------------------------------------------------------------------------
방문지정보
----------------------------------------
Index(['LOTNO_ADDR', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN',
       'REVISIT_INTENTION', 'RCMDTN_INTENTION'],
      dtype='object')
--------------------------------------------------------------------------------

In [ ]:
output_dict['최종'] = output_dict['방문지정보'].copy()

In [ ]:
output_dict['최종']
tmp_df = output_dict['최종']
for col_name in output_dict['최종'].columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")

In [ ]:
tmp_df = output_dict['최종']
# tmp_df = tmp_df.merge(input_dict['동반자정보'], left_index=True, right_index=True)
tmp_df = tmp_df.join(input_dict['동반자정보'], how='inner', on=['TRAVEL_ID'])
# tmp_df.loc['a_a000202', 'COMPANION_AGE_GRP'] = -1
# print(tmp_df.loc['a_a000202'])
print(output_dict['최종'].shape)
print(tmp_df.shape)
print(input_dict['동반자정보'].shape)
print(pd.isna(input_dict['동반자정보']).sum())
print(tmp_df)
print("***"*10)
print(output_dict)

In [ ]:
for keys in input_dict:
    tmp_df = input_dict[keys]
    tmp_col = tmp_df.columns
    # for col_name in tmp_col:
    #     tmp_df[col_name]
    
    if keys=="동반자정보":
        break
    
    print(keys)
    print(output_dict['최종'].shape)
    print(output_dict['최종'].join(tmp_df).shape)
    output_dict['최종'] = output_dict['최종'].join(tmp_df)
    print(output_dict['최종'].columns)
    
        

In [ ]:
tmp_df = output_dict["최종"]
output_index_list = list(set(tmp_df.index.to_list()))
input_index_list = list(set(input_dict["동반자정보"].index.to_list()))
# print(output_index_list)
# print(input_index_list)
# print(input_dict["동반자정보"])
# print(input_dict["동반자정보"].loc["c_c000341"])

for index in output_index_list:
    REL_CD = 0
    COMPANION_AGE_GRP = 0
    if index in input_index_list:
        tmp_len = input_dict["동반자정보"].loc[index].shape
        tmp_data = input_dict["동반자정보"].loc[index]
        if len(tmp_len) >= 2:
            tmp_data = tmp_data.iloc[0].to_frame()
        else:
            tmp_data = tmp_data.to_frame()
        # print(index)
        # print(len(tmp_data))
        # print(tmp_data.to_frame())
        # print(tmp_data)
        # print(tmp_data.values[0,0])
        # print(tmp_data.values[1,0])
        tmp_df.loc[index,"REL_CD"] = tmp_data.values[0,0]
        tmp_df.loc[index,"COMPANION_AGE_GRP"]= tmp_data.values[1,0]
        
output_dict['최종'] = tmp_df

In [ ]:
print(output_dict['최종'].columns)

In [ ]:
tmp_df = output_dict['최종']
for col_name in output_dict['최종'].columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    

tmp_df.to_csv("./data/tmp_dataset.csv")

# 데이터 보정

> LOTNO_ADDR

    885

    필수로 존재해야 하는 값이다.
    없으면 해당 행은 모두 제거

=================
> LODGING_TYPE_CD

    11295

    숙소를 쓰는 경우가 흔하지 않은 것인지  na가 많다
    나머지는 모두 0으로 채운다

=================
> VISIT_AREA_TYPE_CD

    0


=================
> REVISIT_YN

    17
    재방문 할 것인지 여부
    yes - 1
    no - 0으로 변경 후 
    0 값으로 나머지 보정

=================
> DGSTFN

    18
    만족도 점수

    필수 값이다
    평균 값으로 대체

=================
> REVISIT_INTENTION
    
    18
    재방문의향

    필수 값이다
    평균 값으로 대체


=================
> RCMDTN_INTENTION

    18
    추천의향이다

    필수 값이다
    평균 값으로 대체

=================
> SCORE

    18
    위의 값으로 계산된 점수 이다

    필수 값이다
    평균 값으로 대체

=================
> INCOME
    
    2345
    개인 소득 수준이다

    0으로 대체

=================
> GENDER
    
    4003

    2로 대체

=================

> HOUSE_INCOME
    
    4581

    사용x 날림

=================
> TRAVEL_MOTIVE_1

    2348

    

# default로 10:기타로 설정

=================
> TRAVEL_MOTIVE_2
    
    2709
=================
> TRAVEL_MOTIVE_3
    
    3128
=================
> TRAVEL_STYL_1
   
    2345

# 앞의 3개의 값만을 사용
# default로 4:중립 사용

=================
> TRAVEL_STYL_2
    
    2345
=================
> TRAVEL_STYL_3
    
    2345
=================
> TRAVEL_STYL_4
    
    2345
=================
> TRAVEL_STYL_5
    
    2345
=================
> TRAVEL_STYL_6
    
    2345
=================
> TRAVEL_STYL_7
    
    2345
=================
> TRAVEL_STYL_8
    
    2345
=================
> TRAVEL_MISSION
    
    2322
    
# 해당 값은 0으로 초기화 적당한 default 값이 존재하지 않음
=================

=================
> REL_CD

    7368

# 0으로 대체
=================
> COMPANION_AGE_GRP

    7368

# 0으로 대체
=================

In [ ]:
final_df_bk = tmp_df.copy().reset_index()
final_df = final_df_bk.copy()
print(final_df.index)
print(final_df.columns)

In [ ]:
target_col = "LOTNO_ADDR"

print(final_df.shape)
print_isna(final_df[target_col])
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])


In [ ]:
final_df = tmp_df

In [ ]:
target_col = "X_COORD"

print(final_df.shape)
print_isna(final_df[target_col])
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "LODGING_TYPE_CD"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df.shape)
print_isna(tmp_df[target_col])


In [ ]:
final_df = tmp_df

In [ ]:
target_col = "REVISIT_YN"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df.loc[tmp_df[target_col]=="Y", target_col] = 1
tmp_df.loc[tmp_df[target_col]=="N", target_col] = 0

# tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].max())
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col])

o = ((tmp_df[target_col]==1).sum())
z = ((tmp_df[target_col]==0).sum())

print(o+z)

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "DGSTFN"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "REVISIT_INTENTION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "RCMDTN_INTENTION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "SCORE"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "INCOME"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

# tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "HOUSE_INCOME"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df
tmp_df = tmp_df.drop(axis=1, columns=target_col)

print(tmp_df.shape)

In [ ]:
final_df = tmp_df

In [ ]:
print(final_df.shape)
# print_isna(final_df[target_col])

tmp_df = final_df

for i in range(1,4):
    target_col = "TRAVEL_MOTIVE_" + str(i)

    tmp_df[target_col] = tmp_df[target_col].fillna(10)

    print(tmp_df[target_col].head())

    print((tmp_df[target_col]==10).sum())
    print(tmp_df.shape)
    print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
tmp_col = [ c for c in tmp_df.columns if "TRAVEL_STYL" in c]
print(tmp_df[tmp_col].describe())
print(tmp_col)
print(tmp_df[tmp_col].mode(axis=0))
print(tmp_df[tmp_col].mode(axis=1))
print(tmp_df[tmp_col].mode(axis=1)[0].value_counts())
tmp_df['TRAVEL_STYL_0'] = tmp_df[tmp_col].mode(axis=1)[0]


In [ ]:
print(final_df.shape)
# print_isna(final_df[target_col])

tmp_df = final_df

for i in range(0,9):
    target_col = "TRAVEL_STYL_" + str(i)

    print((tmp_df[target_col]==4).sum())
    
    tmp_df[target_col] = tmp_df[target_col].fillna(4)

    print(tmp_df[target_col].head())

    print((tmp_df[target_col]==4).sum())
    print(tmp_df.shape)
    print_isna(tmp_df[target_col])

In [ ]:
tmp_col = [ c for c in tmp_df.columns if "TRAVEL_STYL" in c]
tmp_col.append(tmp_df.columns[-1])
print(tmp_df[tmp_col].describe())
print(tmp_col)
print(tmp_df[tmp_col].mode(axis=0))
print(tmp_df[tmp_col].mode(axis=1))
print(tmp_df[tmp_col].mode(axis=1)[0].value_counts())


In [ ]:
tmp_col = tmp_df.columns
print(tmp_col)
new_col = ['TRAVEL_ID','VISIT_AREA_ID', 'GENDER','LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8', 'TRAVEL_MISSION', 'REL_CD', 'COMPANION_AGE_GRP', 'X_COORD', 'Y_COORD']
tmp_df = tmp_df[new_col]
tmp_col = tmp_df.columns
print(tmp_col)

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "TRAVEL_MISSION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "REL_CD"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "COMPANION_AGE_GRP"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
target_col = "GENDER"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(2)

tmp_df[tmp_df[target_col]=="남"] = 1
tmp_df[tmp_df[target_col]=="여"] = 0

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

In [ ]:
final_df = tmp_df

In [ ]:
tmp_df = final_df.copy()
tmp_df["TRAVEL_MISSION_1"]=0
tmp_df["TRAVEL_MISSION_2"]=0
tmp_df["TRAVEL_MISSION_3"]=0

print(tmp_df['TRAVEL_MISSION'])
# print(tmp_df['TRAVEL_MISSION'].str.split(';').str[0])

tmp_df['TRAVEL_MISSION_1'] = tmp_df['TRAVEL_MISSION'].str.split(';').str[0]
tmp_df['TRAVEL_MISSION_2'] = tmp_df['TRAVEL_MISSION'].str.split(';').str[1]
tmp_df['TRAVEL_MISSION_3'] = tmp_df['TRAVEL_MISSION'].str.split(';').str[2]

# tmp_df[tmp_df['TRAVEL_MISSION_1']==""]['TRAVEL_MISSION_1']
# print(tmp_df["TRAVEL_MISSION_2"])
# print(tmp_df[tmp_df['TRAVEL_MISSION_2']==""]['TRAVEL_MISSION_2'])
# tmp_df = tmp_df['TRAVEL_MISSION_2'].replace("","0")
# print(tmp_df[tmp_df['TRAVEL_MISSION_2']==""]['TRAVEL_MISSION_2'])
# tmp_df[tmp_df['TRAVEL_MISSION_3']==""]['TRAVEL_MISSION_3']

tmp_df['TRAVEL_MISSION_1'] = tmp_df['TRAVEL_MISSION_1'].fillna(0)
tmp_df['TRAVEL_MISSION_2'] = tmp_df['TRAVEL_MISSION_2'].fillna(0)
tmp_df['TRAVEL_MISSION_3'] = tmp_df['TRAVEL_MISSION_3'].fillna(0)


In [ ]:
final_df = tmp_df

In [ ]:
tmp_df = final_df
print(tmp_df.columns)
for col_name in final_df.columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    
new_col = ['TRAVEL_ID','VISIT_AREA_ID', 'GENDER','INCOME', 'TRAVEL_MISSION', 'REL_CD', 'LODGING_TYPE_CD',
        'TRAVEL_MISSION_1', 'TRAVEL_MISSION_2', 'TRAVEL_MISSION_3',         
        'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
        'TRAVEL_STYL_0', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
        'TRAVEL_STYL_5', 'TRAVEL_STYL_6','TRAVEL_STYL_7', 'TRAVEL_STYL_8',
        'COMPANION_AGE_GRP',
        'REVISIT_YN', 'DGSTFN','REVISIT_INTENTION', 'RCMDTN_INTENTION','SCORE',
        'LOTNO_ADDR',  'VISIT_AREA_TYPE_CD', 'X_COORD', 'Y_COORD']
tmp_df = tmp_df[new_col]
tmp_df.to_csv("./travel_dataset.csv")

In [ ]:
tmp_df = pd.read_csv('./travel_dataset.csv')
tmp_df = tmp_df.fillna(0)
print(tmp_df.columns)
for col_name in final_df.columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    
tmp_df.to_csv("./travel_dataset.csv")
    